## 循环神经网络 RNN

&emsp;&emsp;在潜变量自回归模型中，我们采用潜变量$h_{t}$总结过去的信息。

$$
p(h_{t} | h_{t-1}, x_{t-1}) \\
p(x_{t} | h_{t},x_{t-1})
$$

&emsp;&emsp;接下来就是说如何把这个模型做成`RNN`:

1. 针对$p(h_{t} | h_{t-1}, x_{t-1})$这一项, 我们可以建模成：

$$
h_{t} = \tanh(W_{hh}h_{t-1} + W_{hx}x_{t-1} + b_{h})
$$

&emsp;&emsp;上式中，如果去掉$W_{hh}h_{t-1}$这一项的话，就会退化成`MLP`了。

2. 针对$p(x_{t} | h_{t},x_{t-1})$这一项，我们的输出可以描述为:

$$
o_{t} = \phi(W_{ho}h_{t} + b_{o})
$$

## torch.nn.RNN

在`PyTorch`中，`RNN`的`API`地址为：[torch.nn.RNN(*args, **kwargs)](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html?highlight=rnn#torch.nn.RNN)

其需要的参数有:

1. `input_size`: 输入数据$x$的特征大小。
2. `hidden_size`: 隐藏状态$h$的特征大小。
3. `num_layers`: 默认只有一层，可以通过改变`num_layers`改变层数的大小。
4. `nonlinearity`: 非线性激活函数，可以为`tanh`或者`relu`，默认为`tanh`。
5. `bias`: 如果设置为`False`的话，就没有`bias`，默认为`True`。
6. `batch_first`: 若设置为`True`，则提供的数据维度为`(batch, seq, feature)`，否则为`(seq, batch, feature)`，默认为`False`。
7. `dropout`：如果为非`0`的值，则会设置`dropout`，默认为`0`。
8. `bidirectional`: 是否为双向。若为双向的结构，则输出的结果大小为 `2 * hidden_size`。


实例化之后，输入数据有两部分：`input,h_0`。输入数据维度为：$(L, N, H_{in})$, 如果`batch_first=False`, 则维度为：$(N, L, H_{in})$。`h_0`的数据`shape`为`(D * num_layers, N, H_out)`。

其中：$N=batch\ size, L=sequence\ length, D = 2 \ if\ bidrectional=True,\  otherwise = 1$, $H_{in}=input\_size, H_{out}=hidden\_size$。

In [1]:
import torch
import torch.nn as nn

bs, T = 2, 3 # 批大小，输入序列长度
input_size, hidden_size = 2, 3  # 输入特征大小，隐含层特征大小

input_data = torch.randn(bs, T, input_size)  # 随机初始化输入特征数据。


### 单向，单层

#### 双向，单层的API调用

In [2]:
h_prev = torch.zeros(bs, hidden_size)  # 初始化隐含状态
single_rnn = nn.RNN(input_size, hidden_size, batch_first=True)  # 输入特征维度为4， 输出特征维度为3。
output, hn = single_rnn(input_data, h_prev.unsqueeze(0))  # 不指定h_0的话，则默认会给0。
print(output)
print(hn)

tensor([[[-0.0658, -0.7007, -0.7684],
         [-0.7843, -0.8803, -0.1957],
         [-0.7789, -0.8376, -0.1086]],

        [[-0.1183, -0.9732, -0.8664],
         [-0.7132, -0.8307, -0.3912],
         [-0.5061, -0.8704, -0.5720]]], grad_fn=<TransposeBackward1>)
tensor([[[-0.7789, -0.8376, -0.1086],
         [-0.5061, -0.8704, -0.5720]]], grad_fn=<StackBackward0>)


从上述输出结果可以看到，输出结果为`batch size`为`1`，序列长度为`2`， 每个序列的输出的维度都是`3`，最后一个序列的输出就是`hn`。

#### 双向，单层RNN前向传播理解

In [20]:
def rnn_forward(input_data, weight_ih, bias_ih, weight_hh, bias_hh, h_prev):
    bs, T, input_size = input_data.shape
    h_dim = weight_ih.shape[0]
    h_out = torch.zeros(bs, T, h_dim)  # 初始化一个输出(状态)矩阵
    
    for t in range(T):
        x = input_data[:, t, :].unsqueeze(2)  # 获取当前时刻输入特征， bs * input_size * 1
        
        w_ih_batch = weight_ih.unsqueeze(0).tile(bs, 1, 1)  # bs * h_dim * input_size
        w_hh_batch = weight_hh.unsqueeze(0).tile(bs, 1, 1)  # bs * h_dim * h_dim
        
        w_times_x = torch.bmm(w_ih_batch, x).squeeze(-1)  # bs * h_dim
        w_times_h = torch.bmm(w_hh_batch, h_prev.unsqueeze(2)).squeeze(-1)  # bs * h_dim
        
        h_prev = torch.tanh(w_times_x + bias_ih + w_times_h + bias_hh)
        
        h_out[:,t,:] = h_prev
    
    return h_out, h_prev.unsqueeze(0)  # 单向单层的，但是官方给的是三维的。

In [21]:
for k, v in single_rnn.named_parameters():
    print(k, v)

weight_ih_l0 Parameter containing:
tensor([[ 0.0172,  0.3831],
        [ 0.5762, -0.0787],
        [ 0.1433, -0.4506]], requires_grad=True)
weight_hh_l0 Parameter containing:
tensor([[-0.1881,  0.3242, -0.2890],
        [ 0.2778,  0.1097,  0.4763],
        [-0.0576,  0.1187,  0.2664]], requires_grad=True)
bias_ih_l0 Parameter containing:
tensor([-0.2626, -0.4763, -0.2100], requires_grad=True)
bias_hh_l0 Parameter containing:
tensor([-0.4453, -0.2321, -0.0434], requires_grad=True)


In [22]:
output, hn = rnn_forward(input_data, single_rnn.weight_ih_l0, single_rnn.bias_ih_l0, \
                         single_rnn.weight_hh_l0, single_rnn.bias_hh_l0, h_prev)
print(output)
print(hn)

RuntimeError: batch2 must be a 3D tensor

可以看到输出结果与官方`API`的调用结果一致。

### 双向，单层

#### 双向，单层的API调用

In [6]:
bidirectional_rnn = nn.RNN(input_size, hidden_size, batch_first=True, bidirectional=True)  # 输入特征维度为4， 输出特征维度为3， 1层。
h_prev = torch.zeros(2, bs, hidden_size)  # 初始化隐含状态
bi_output, bi_hn = bidirectional_rnn(input_data, h_prev)  # 不指定h_0的话，则默认会给0。
print(bi_output)
print(bi_hn)

tensor([[[-0.2765,  0.2758, -0.1308,  0.0694, -0.5915,  0.9058],
         [-0.2322, -0.4363, -0.6479,  0.3023, -0.2813,  0.6778],
         [-0.5839, -0.5484, -0.3730,  0.0880, -0.4876,  0.6034]],

        [[ 0.0128, -0.5697, -0.5231,  0.2691, -0.1512,  0.9403],
         [-0.6673, -0.4781, -0.3145,  0.3161, -0.4270,  0.7228],
         [-0.2692, -0.1458,  0.0146,  0.0541, -0.5821,  0.7424]]],
       grad_fn=<TransposeBackward1>)
tensor([[[-0.5839, -0.5484, -0.3730],
         [-0.2692, -0.1458,  0.0146]],

        [[ 0.0694, -0.5915,  0.9058],
         [ 0.2691, -0.1512,  0.9403]]], grad_fn=<StackBackward0>)


#### 双向，单层RNN前向传播理解

双向`RNN`的前向计算过程是什么样的呢？

In [7]:
def bidirectional_rnn_forward(input_data, weight_ih, bias_ih, weight_hh, bias_hh, h_prev,\
                             weight_ih_reverse, weight_hh_reverse, bias_ih_reverse, bias_hh_reverse, \
                              h_prev_reverse):
    
    bs, T, input_size = input_data.shape
    h_dim = weight_ih.shape[0]
    h_out = torch.zeros(bs, T, h_dim * 2)  # 初始化一个输出(状态)矩阵, 双向是两倍特征大小。
    
    forward_output = rnn_forward(input_data, weight_ih, bias_ih, weight_hh, bias_hh, h_prev)[0]
    
    backward_output = rnn_forward(torch.flip(input_data, [1]), weight_ih_reverse, weight_hh_reverse, \
                                  bias_ih_reverse, bias_hh_reverse, h_prev_reverse)[0]  # backward layer
    
    h_out[:, :, :h_dim] = forward_output
    h_out[:, :, h_dim:] = backward_output

    return h_out, h_out[:, -1, :].reshape((bs, 2, h_dim)).transpose(0, 1)

In [8]:
for k, v in bidirectional_rnn.named_parameters():
    print(k, v)

weight_ih_l0 Parameter containing:
tensor([[-0.1343,  0.0687],
        [ 0.4123,  0.2929],
        [ 0.1976,  0.2232]], requires_grad=True)
weight_hh_l0 Parameter containing:
tensor([[-0.4889,  0.2893,  0.3699],
        [-0.2880,  0.4017,  0.0363],
        [-0.3799, -0.3433, -0.0959]], requires_grad=True)
bias_ih_l0 Parameter containing:
tensor([ 0.0991, -0.4202, -0.0296], requires_grad=True)
bias_hh_l0 Parameter containing:
tensor([-0.5049,  0.2322, -0.4667], requires_grad=True)
weight_ih_l0_reverse Parameter containing:
tensor([[-0.0778, -0.0266],
        [-0.2008, -0.1091],
        [-0.1094,  0.2378]], requires_grad=True)
weight_hh_l0_reverse Parameter containing:
tensor([[-0.5555, -0.1929,  0.2554],
        [ 0.5047, -0.5450, -0.2321],
        [ 0.3033,  0.1103,  0.3149]], requires_grad=True)
bias_ih_l0_reverse Parameter containing:
tensor([-0.3708, -0.2364,  0.4141], requires_grad=True)
bias_hh_l0_reverse Parameter containing:
tensor([ 0.4217, -0.4190,  0.4108], requires_grad=True

In [9]:
output,hn = bidirectional_rnn_forward(input_data, bidirectional_rnn.weight_ih_l0, bidirectional_rnn.bias_ih_l0,\
                         bidirectional_rnn.weight_hh_l0, bidirectional_rnn.bias_hh_l0, h_prev[0], \
                         bidirectional_rnn.weight_ih_l0_reverse, bidirectional_rnn.bias_ih_l0_reverse, \
                         bidirectional_rnn.weight_hh_l0_reverse, bidirectional_rnn.bias_hh_l0_reverse, h_prev[1])
print(output)
print(hn)

tensor([[[-0.2765,  0.2758, -0.1308,  0.0880, -0.4876,  0.6034],
         [-0.2322, -0.4363, -0.6479,  0.3023, -0.2813,  0.6778],
         [-0.5839, -0.5484, -0.3730,  0.0694, -0.5915,  0.9058]],

        [[ 0.0128, -0.5697, -0.5231,  0.0541, -0.5821,  0.7424],
         [-0.6673, -0.4781, -0.3145,  0.3161, -0.4270,  0.7228],
         [-0.2692, -0.1458,  0.0146,  0.2691, -0.1512,  0.9403]]],
       grad_fn=<CopySlices>)
tensor([[[-0.5839, -0.5484, -0.3730],
         [-0.2692, -0.1458,  0.0146]],

        [[ 0.0694, -0.5915,  0.9058],
         [ 0.2691, -0.1512,  0.9403]]], grad_fn=<TransposeBackward0>)


### RNNCell

`RNN`其实就是多次的`RNNCell`的计算。

函数原型为:[torch.nn.RNNCell](https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html?highlight=rnncell#torch.nn.RNNCell)

```python
CLASS torch.nn.RNNCell(input_size, hidden_size, bias=True, nonlinearity='tanh', device=None, dtype=None)
```

举例为:

```python
rnn = nn.RNNCell(10, 20)
input = torch.randn(6, 3, 10)
hx = torch.randn(3, 20)
output = []
for i in range(6):
    hx = rnn(input[i], hx)
    output.append(hx)
```


## Vanilla-RNN-代码

### Numpy实现

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import time
sns.set()

def get_vocab(file, lower = False):
    with open(file, 'r') as fopen:
        data = fopen.read() # 将文件中的所有数据读取进来。
    if lower:
        data = data.lower()
    
    vocab = list(set(data))
    return data, vocab

def embed_to_control(data, vocab):
    onehot = np.zeros((len(data), len(vocab)), dtype = np.float32)
    for i in range(len(data)):
        onehot[i, vocab.index(data[i])] = 1.0
    return onehot

In [11]:
text, text_vocab = get_vocab('../consumer.h', lower = False)
one_hot = embed_to_control(text, text_vocab)
print('len text: ', len(text))
print('len text_vocab: ', len(text_vocab))
print('one_hot shape: ', one_hot.shape)

len text:  15294
len text_vocab:  75
one_hot shape:  (15294, 75)


&emsp;&emsp;`tanh`激活函数为:

$$
\tanh x=\frac{\sinh x}{\cosh x}=\frac{e^{x}-e^{-x}}{e^{x}+e^{-x}}
$$

&emsp;&emsp;导数为:

$$
(\tanh x)^{\prime}=\operatorname{sech}^{2} x=1-\tanh ^{2} x
$$

In [12]:
def tanh(x, grad=False):
    if grad:
        output = np.tanh(x)
        return (1.0 - np.square(output))
    else:
        return np.tanh(x)

$$
h_{t} = \tanh(W_{hh}h_{t-1} + W_{hx}x_{t-1} + b_{h})
$$

$$
o_{t} = \phi(W_{ho}h_{t} + b_{o})
$$

In [13]:
def forward_rnn_recurrent(x, prev_state, W_hx, W_hh, W_ho):
    mul_hx = np.dot(x, W_hx.T)
    
    # 处理隐藏状态部分。
    mul_hh = np.dot(prev_state, W_hh.T)
    add_previous_now = mul_hx + mul_hh
    current_state = tanh(add_previous_now)
    
    # 处理输出部分。
    mul_o = np.dot(current_state, W_ho.T)
    return (mul_hx, mul_hh, add_previous_now, current_state, mul_o)

In [14]:
def softmax(x):
    """
    x: np.max(x)取的是二维数组x中的最大值。
    """
    exp_scores = np.exp(x - np.max(x))
    return exp_scores / (np.sum(exp_scores, axis=1, keepdims=True) + 1e-8)

&emsp;&emsp;多分类的交叉熵损失如下:

$$
L=\frac{1}{N} \sum_{i} L_{i}=-\frac{1}{N} \sum_{i} \sum_{c=1}^{M} y_{i c} \log \left(p_{i c}\right)
$$

&emsp;&emsp;其中$M$表示类别的数量，$y_{ic}$是符号函数(0或者1), 如果样本$i$的真实类别等于$c$取1，否者取0。$p_{ic}$表示观测样本$i$属于类别$c$的概率。

In [15]:
def cross_entropy(Y_hat, Y, epsilon=1e-12):
    Y_hat = np.clip(Y_hat, epsilon, 1. - epsilon)
    N = Y_hat.shape[0]
    return -np.sum(np.sum(Y * np.log(Y_hat + 1e-9))) / N

&emsp;&emsp;`RNN`的反向传播:

In [16]:
def backward_multiply_gate(w, x, dz):
    """
    w shape = (75, 128)
    x shape = (64, 128)
    dz shape = (64, 75)
    """
    dw = np.dot(dz.T, x) # shape = (75, 128)
    dx = np.dot(w.T, dz.T) # shape = (128, 64)
    return dw, dx

def backward_add_gate(x1, x2, dz):
    dx1 = dz * np.ones_like(x1)
    dx2 = dz * np.ones_like(x2)
    return dx1, dx2

def backward_rnn_recurrent(x, prev_state, W_hx, W_hh, W_ho, d_mu_o, saved_graph):
    mul_hx, mul_hh, add_previous_now, current_state, mul_o = saved_graph
    
    dW_ho, d_CurrentState = backward_multiply_gate(W_ho, current_state, d_mu_o)
    
    dadd_previous_now = tanh(add_previous_now, True) * d_CurrentState.T
    
    dmul_hh, dmul_hx = backward_add_gate(mul_hh, mul_hx, dadd_previous_now)
    dW_hh, dprev_state = backward_multiply_gate(W_hh, prev_state, dmul_hh)
    dW_hx, dx = backward_multiply_gate(W_hx, x, dmul_hx)
    
    return (dprev_state, dW_hx, dW_hh, dW_ho)

In [17]:
epoch = 1000
learning_rate = 0.0001
batch_size = 64
sequence_length = int(12)
dimension = one_hot.shape[1]
print('dimension is :', dimension)
possible_batch_id = range(len(text) - sequence_length - 1)
hidden_dim = 128

W_hx = np.random.randn(hidden_dim, dimension) / np.sqrt(hidden_dim)
W_hh = np.random.randn(hidden_dim, hidden_dim) / np.sqrt(hidden_dim)
W_ho = np.random.randn(dimension, hidden_dim) / np.sqrt(hidden_dim)

for i in range(epoch):
    batch_x = np.zeros((batch_size, sequence_length, dimension))
    batch_y = np.zeros((batch_size, sequence_length, dimension))
    
    batch_id = random.sample(possible_batch_id, batch_size)  # 随机采样，选择batch_id。
    prev_s = np.zeros((batch_size, hidden_dim))
    
    for n in range(sequence_length):
        id1 = [k + n for k in batch_id]
        id2 = [k + n + 1 for k in batch_id]
        
        batch_x[:, n, :] = one_hot[id1]
        batch_y[:, n, :] = one_hot[id2]
    
    layers = []
    out_logits = np.zeros((batch_size, sequence_length, dimension))
    for n in range(sequence_length):
        layers.append(forward_rnn_recurrent(batch_x[:, n, :], prev_s, W_hx, W_hh, W_ho))
        prev_s = layers[-1][3]
        out_logits[:, n, :] = layers[-1][-1]
    
    probs = softmax(out_logits.reshape((-1, dimension)))
    y = np.argmax(batch_y.reshape((-1, dimension)), axis=1)
    accuracy = np.mean(np.argmax(probs, axis=1) == y)
    
    loss = cross_entropy(probs, batch_y.reshape((-1, dimension)))
    
    # 之后需要开始计算反向更新部分了。
    
    # 1. 计算梯度:
    delta = probs  # 取网络的输出结果。
    delta[range(y.shape[0]), y] -= 1  # 将网络输出结果中对应标签的那个概率减去1。
    delta = delta.reshape((batch_size, sequence_length, dimension))
    
    dW_hx = np.zeros(W_hx.shape)
    dW_hh = np.zeros(W_hh.shape)
    dW_ho = np.zeros(W_ho.shape)
    prev_state = np.zeros((batch_size, hidden_dim))
    
    for n in range(sequence_length):
        d_mul_o = delta[:, n, :] # shape = (batch_size, dimension)
        
        dprev_s, dW_hx_t, dW_hh_t, dW_ho_t = backward_rnn_recurrent(batch_x[:,n,:], prev_state, 
                                                         W_hx, W_hh, W_ho, d_mul_o, layers[n])
        prev_state = layers[n][3]
        dW_hx += dW_hx_t
        dW_hh += dW_hh_t
        dW_ho += dW_ho_t
    
    # 更新
    W_hx -= learning_rate * dW_hx 
    W_hh -= learning_rate * dW_hh
    W_ho -= learning_rate * dW_ho
    if(i + 1) % 50 == 0:
        print("epoch {}, loss {}, accuracy {}".format(i+1, loss, accuracy))

dimension is : 75
epoch 50, loss 4.18223108646421, accuracy 0.11197916666666667
epoch 100, loss 3.842034400735851, accuracy 0.22005208333333334
epoch 150, loss 3.541025696688905, accuracy 0.21614583333333334
epoch 200, loss 3.3996178844139746, accuracy 0.22395833333333334
epoch 250, loss 3.3777040144544883, accuracy 0.25390625
epoch 300, loss 3.186191625076074, accuracy 0.3203125
epoch 350, loss 3.0572455963409877, accuracy 0.3046875
epoch 400, loss 3.059383976735328, accuracy 0.30078125
epoch 450, loss 3.012591920136936, accuracy 0.3268229166666667
epoch 500, loss 2.7137002191439987, accuracy 0.37890625
epoch 550, loss 2.726418368790578, accuracy 0.359375
epoch 600, loss 2.7612495795096694, accuracy 0.3723958333333333
epoch 650, loss 2.6242703595614034, accuracy 0.3841145833333333
epoch 700, loss 2.408088679008324, accuracy 0.45703125
epoch 750, loss 2.4681698348737746, accuracy 0.4205729166666667
epoch 800, loss 2.3893067146578, accuracy 0.453125
epoch 850, loss 2.3606142764147546, a

### Pytorch实现

In [18]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        """
        """
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True, nonlinearity='tanh')
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_dim))
        out, hn = self.rnn(x, None)
        
        out = self.fc(out)
        return out
rnn = RNN(input_dim = dimension, hidden_dim = hidden_dim, num_layers=1, output_dim = dimension)
print(rnn)

RNN(
  (rnn): RNN(75, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=75, bias=True)
)


In [19]:
# 分开定义softmax运算和交叉熵损失函数会造成数值不稳定。
# 因此PyTorch提供了一个具有良好数值稳定性且包括softmax运算和交叉熵计算的函数。
criterion = nn.CrossEntropyLoss()

learning_rate = 0.5
optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate)

for i in range(epoch):
    batch_x = np.zeros((batch_size, sequence_length, dimension), dtype=np.float32)
    batch_y = np.zeros((batch_size, sequence_length, dimension), dtype=np.float32)
    batch_id = random.sample(possible_batch_id, batch_size)  # 随机采样，选择batch_id。
    prev_s = np.zeros((batch_size, hidden_dim))
    for n in range(sequence_length):
        id1 = [k + n for k in batch_id]
        id2 = [k + n + 1 for k in batch_id]
        
        batch_x[:, n, :] = one_hot[id1]
        batch_y[:, n, :] = one_hot[id2]
    
    # 从Numpy转成torch之后送入神经网络中去。
    output = rnn(torch.from_numpy(batch_x))  # torch.Size([64, 12, 75])
    label = torch.argmax(torch.from_numpy(batch_y).view(-1, dimension), dim=1) # shape = 786
    
    accuracy = np.mean(torch.argmax(output.view(-1, dimension), axis=1).numpy() == label.numpy())
    
    optimizer.zero_grad()
    loss = criterion(output.view(-1, dimension), label)
    loss.backward()
    optimizer.step()
    
    if(i + 1) % 50 == 0:
        print("epoch {}, loss {}, accuracy {}".format(i+1, loss.item(), accuracy))
    

epoch 50, loss 3.401153564453125, accuracy 0.11328125
epoch 100, loss 3.234724998474121, accuracy 0.20833333333333334
epoch 150, loss 2.519819498062134, accuracy 0.3919270833333333
epoch 200, loss 2.083822011947632, accuracy 0.4622395833333333
epoch 250, loss 1.9139952659606934, accuracy 0.5377604166666666
epoch 300, loss 1.7888798713684082, accuracy 0.5533854166666666
epoch 350, loss 1.6153966188430786, accuracy 0.5963541666666666
epoch 400, loss 1.2576789855957031, accuracy 0.6861979166666666
epoch 450, loss 1.2486813068389893, accuracy 0.6822916666666666
epoch 500, loss 1.3055974245071411, accuracy 0.66015625
epoch 550, loss 1.154812216758728, accuracy 0.70703125
epoch 600, loss 1.0780938863754272, accuracy 0.7317708333333334
epoch 650, loss 0.9139544367790222, accuracy 0.7669270833333334
epoch 700, loss 1.019874930381775, accuracy 0.7369791666666666
epoch 750, loss 1.0062402486801147, accuracy 0.7369791666666666
epoch 800, loss 0.7421910762786865, accuracy 0.8125
epoch 850, loss 0.